In [ ]:
# pip install xlrd==1.2.0

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import itertools
from collections import Counter

import xlrd
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True
xls = xlrd.open_workbook(r'output_folder/CC_output_20100224.xlsx', on_demand=True)


C:\Users\czhao\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\czhao\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\czhao\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
sheets_names = xls.sheet_names()

In [3]:
def is_valid_chunk(df_slice):
        # Check if at least one row in the slice has 'number_of_words' >= 3
        # and non-empty col
        valid_rows = df_slice[
            (df_slice['number_of_words'] >= 3) &
            (df_slice[col].astype(bool))
        ]
        if valid_rows.empty:
            return False
        return True

# calculate the min distance for given sequence
def calculate_min_seq_dist(sequence):
        dist = 0
        sequence = list(itertools.chain.from_iterable(sequence))
        for idx, val in enumerate(sequence[:-1]):
            dist += abs(val - sequence[idx+1])
        return dist

def non_empty(sequence):
    count = 0
    for val in sequence[:-1]:
        if val != []:
            count +=1
    return count

def merge_to_tuples(list1, list2):
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list
 

#     find the start point for forward and backward
def find_starting_point(chunk_df):
    max_words = chunk_df[chunk_df[col].astype(bool)]['number_of_words'].max()
    starting_point_found = False
    correct_sent_num = None
    correct_row = None
    min_dist = 100000000

    while not starting_point_found and max_words >= 3:
        valid_rows = chunk_df[(chunk_df['number_of_words'] == max_words) & (chunk_df[col].astype(bool))]
        if not valid_rows.empty and len(valid_rows.iloc[0][col]) > 1:
            best_sequence = []
            for sent_num in valid_rows.iloc[0][col]:
                sequence = compute_sequence(chunk_df, sent_num, valid_rows.index.values[0])

                if non_empty(sequence) == non_empty(best_sequence):
                    if calculate_min_seq_dist(sequence) < calculate_min_seq_dist(best_sequence):
                        best_sequence = sequence
                        correct_sent_num = [sent_num]
                        correct_row = [valid_rows.index.values[0]]
                    elif calculate_min_seq_dist(sequence) == calculate_min_seq_dist(best_sequence):
                        best_sequence = sequence
                        correct_sent_num.append(sent_num)
                        correct_row.append(valid_rows.index.values[0])

                elif non_empty(sequence) > non_empty(best_sequence):    
                    best_sequence = sequence
                    correct_sent_num = [sent_num]
                    correct_row = [valid_rows.index.values[0]]

            starting_point_found = True
        elif not valid_rows.empty:
            correct_sent_num = [valid_rows.iloc[0][col][0]]
            correct_row = [valid_rows.index.values[0]]
            starting_point_found = True
        else:
            max_words -= 1
            
    return zip(*list(set(merge_to_tuples(correct_sent_num, correct_row))))

    
# get exact longest sequences
def compute_sequence(chunk_df, initial_sent_num, initial_sent_row):
    last_correct_sent_num = initial_sent_num
    sequence_forward = []
    sequence_backward = []


    for index, row in chunk_df.loc[initial_sent_row:,:].iterrows():
        current_values = row[col]
        if not current_values:
            sequence_forward.append((index, []))
            continue

        valid_values = [x for x in current_values if x >= last_correct_sent_num]

        if valid_values:
            min_distance = min(abs(x - last_correct_sent_num) for x in valid_values)
            if min_distance <= 10:
                new_correct = min(x for x in valid_values if abs(x - last_correct_sent_num) == min_distance)
                last_correct_sent_num = new_correct

                sequence_forward.append((index, new_correct))
            else:
                sequence_forward.append((index, []))

    last_correct_sent_num = initial_sent_num

    for index, row in chunk_df.loc[:initial_sent_row-1,:].iloc[::-1].iterrows():
        current_values = row[col]
        if not current_values:
            sequence_backward.insert(0, (index, []))
            continue

        valid_values = [x for x in current_values if x <= last_correct_sent_num]
        if valid_values:

            min_distance = min(abs(x - last_correct_sent_num) for x in valid_values)

            if min_distance <= 10:
                new_correct = min(x for x in valid_values if abs(x - last_correct_sent_num) == min_distance)
                last_correct_sent_num = new_correct
                sequence_backward.insert(0, (index, new_correct))
            else:
                sequence_backward.insert(0, (index, []))

    merged_dict = {i:[] for i in range(start, end)}


    # Insert values from list1 into merged_dict
    for index, value in sequence_forward:
        if value != [] and value not in merged_dict[index]:
            merged_dict[index].append(value)

    # Insert values from list2 into merged_dict
    for index, value in sequence_backward:
        if value != [] and value not in merged_dict[index]:
            merged_dict[index].append(value)

    # Convert merged_dict back to list of (index, value) pairs
    merged_list = [values for index, values in merged_dict.items()]

    return merged_list


In [4]:
df = pd.read_excel('output_folder/CC_output_20100224.xlsx') 
df_transcript = pd.read_excel(open('output_folder/CC_output_20100224.xlsx', 'rb'),
              sheet_name='Transcript') 

proc_cols = ['occurrence_sent_num', 'timed_matches']

In [5]:
for col in proc_cols:
    df[col] = df[col].replace({np.nan: '[]'})
    df[col] = df.apply(lambda x: literal_eval(x[col]), axis=1)

    chunks = []
    i = 0
    while i < len(df):
        count_empty = 0
        j = i
        while j < len(df) and (count_empty <= 2):
            if not df.iloc[j][col]:
                count_empty += 1
            else:
                count_empty = 0
            j += 1
        # If the chunk has at least 5 rows and is valid
        if (j - i) >= 5 and is_valid_chunk(df.iloc[i:j]):
            chunks.append((i, j))
            i = j
        else:
            i += 1

    for start, end in chunks:

        chunk_df = df.iloc[start:end].copy()

        correct_sent_num, correct_row = find_starting_point(chunk_df)

        longest_sequence_series_ls = []

        for (i, j) in zip(correct_sent_num, correct_row):
            # Compute the sequence with the longest valid set of timed_matches values
            longest_sequence = compute_sequence(chunk_df, i, j)

            occurred_flag = 0
            modified_longest_sequence = []
            for idx, val in enumerate(longest_sequence):
                if val!= []:
                    occurred_flag = 1
                    if idx > 0 and idx < len(longest_sequence)-1:
                        if longest_sequence[idx-1]==[] and longest_sequence[idx+1]==[]:
                            modified_longest_sequence.append(chunk_df[col].iloc[idx])
                        else:
                            modified_longest_sequence.append(val)
                    elif idx == 0:
                        if longest_sequence[1] == []:
                            modified_longest_sequence.append(chunk_df[col].iloc[idx])
                        else:
                            modified_longest_sequence.append(val)
                    elif idx == len(longest_sequence)-1:
                        if longest_sequence[len(longest_sequence)-2] == []:
                            modified_longest_sequence.append(chunk_df[col].iloc[idx])
                        else:
                            modified_longest_sequence.append(val)                   

                else: 
                    occurred_flag = 0
                    modified_longest_sequence.append(val)

            longest_sequence_series = modified_longest_sequence
            if not bool(longest_sequence_series_ls):
                longest_sequence_series_ls = longest_sequence_series
            else:
                longest_sequence_series_ls = [sublist1 + sublist2 for sublist1, sublist2 in zip(longest_sequence_series_ls, longest_sequence_series)]


        df.loc[start:end-1, 'Longest_Sequence_' + col] = pd.Series(longest_sequence_series_ls, index=df.iloc[start:end].index)
        
        

In [6]:
def get_frag_in_elapse_time(df, based_col, new_col, gap_time=35):
    # Assuming you have a DataFrame named 'df' with appropriate columns

    last_sent = 0
    next_sent = 0
    elapse_time0 = 0
#     gap_time = 60
    
    df[new_col] = pd.Series(dtype='int')

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        frag_count_gt1 = row[based_col]
        time_span = row['time_span']

        next_sent = index
        if frag_count_gt1 > 0:
            # Set indicator = 1 and update last_sent
            df.at[index, new_col] = 1
            last_sent = index
            elapse_time0 = 0
        else:
            if last_sent == 0:
                # Set indicator = 0 if no matches found yet
                df.at[index, new_col] = 0
            else:

                elapse_time0 += time_span

                if elapse_time0 <= gap_time:
                    # Set indicator = 1 if time span <= gap_time
                    df.at[index, new_col] = 1
                else:
                    # Set indicator = 0 for rows between last_sent+1 and next_sent-1
                    df.loc[last_sent + 1:next_sent, new_col] = 0
                    elapse_time0 = 0
                    last_sent=0

In [26]:
# recalculate frag for Longest_Sequence_occurrence_sent_num and Longest_Sequence_timed_matches

for col in ['Longest_Sequence_occurrence_sent_num', 'Longest_Sequence_timed_matches']:

    df.loc[df[col].isnull(),[col]] = df.loc[df[col].isnull(),col].apply(lambda x: [])

    occurrence_dic = Counter(list(itertools.chain.from_iterable(df[df['cleaned_text'].str.len()>1][df[col].apply(lambda x: len(x)==1)][col].values)))

    df_transcript['Frag_count_gt'+'_'+col] = df_transcript['sentence_num'].apply(lambda x: occurrence_dic[x] if x in occurrence_dic else 0)

    get_frag_in_elapse_time(df_transcript, 'Frag_count_gt'+'_'+col, 'Frag_in_elasp_time'+'_'+col)

    # Add two dummy columns
    df_transcript['dummy_preceding'+'_'+col] = 0
    df_transcript['dummy_subsequent'+'_'+col] = 0

    # Iterate over each row in the DataFrame
    for index, row in df_transcript.iterrows():
        # Calculate the sum of the preceding 5 sentences
        if index > 4:
            preceding_sum = df_transcript['Frag_in_elasp_time'+'_'+col][index - 5:index].sum()
        else:
            preceding_sum = df_transcript['Frag_in_elasp_time'+'_'+col][0:index].sum()

        # Calculate the sum of the subsequent 5 sentences or till the end of the file
        if index < len(df) - 5:
            subsequent_sum = df_transcript['Frag_in_elasp_time'+'_'+col][index + 1:index + 6].sum()
        else:
            subsequent_sum = df_transcript['Frag_in_elasp_time'+'_'+col][index + 1:].sum()

        # Set dummy variables
        df_transcript.at[index, 'dummy_preceding'+'_'+col] = 1 if preceding_sum > 0 else 0
        df_transcript.at[index, 'dummy_subsequent'+'_'+col] = 1 if subsequent_sum > 0 else 0

    # Set the values in the third column based on the conditions
    df_transcript['Frag_range5'+'_'+col] = 0

    for index, row in df_transcript.iterrows():
        second_value = row['Frag_in_elasp_time_'+col]
        dummy_preceding = row['dummy_preceding'+'_'+col]
        dummy_subsequent = row['dummy_subsequent'+'_'+col]

        if second_value == 0:
            df_transcript.at[index, 'Frag_range5'+'_'+col] = 0
        elif second_value == 1 and dummy_preceding == 0 and dummy_subsequent == 0:
            df_transcript.at[index, 'Frag_range5'+'_'+col] = 0
        else:
            df_transcript.at[index, 'Frag_range5'+'_'+col] = 1

    # Drop the dummy columns if they are no longer needed
    df_transcript = df_transcript.drop(['dummy_preceding'+'_'+col, 'dummy_subsequent'+'_'+col], axis=1)


C:\Users\czhao\AppData\Local\Temp\ipykernel_2720\3695093394.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  occurrence_dic = Counter(list(itertools.chain.from_iterable(df[df['cleaned_text'].str.len()>1][df[col].apply(lambda x: len(x)==1)][col].values)))
C:\Users\czhao\AppData\Local\Temp\ipykernel_2720\3695093394.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  occurrence_dic = Counter(list(itertools.chain.from_iterable(df[df['cleaned_text'].str.len()>1][df[col].apply(lambda x: len(x)==1)][col].values)))


In [27]:
with pd.ExcelWriter("output_folder/CC_output_20100224.xlsx") as writer:

    df.to_excel(writer, sheet_name=sheets_names[0], index=False)
    df_transcript.to_excel(writer, sheet_name=sheets_names[1], index=False)


In [7]:

# # Recursive function to get all sequences with their gaps, not used because computation limitations
# def get_all_sequences(numbers, index=0, prev_value=None):
#     if index == len(numbers):
#         return [([], 0)]
    
#     # Skip the row if the list is empty
#     if not numbers[index]:
#         return [( [None] + seq, gap) for seq, gap in get_all_sequences(numbers, index + 1, prev_value)]
    
#     sequences = []
    
#     for value in numbers[index]:
#         if prev_value is not None:
#             current_gap = abs(value - prev_value)
#         else:
#             current_gap = 0
        
#         for subsequent_seq, subsequent_gap in get_all_sequences(numbers, index + 1, value):
#             sequences.append(([value] + subsequent_seq, current_gap + subsequent_gap))
    
#     return sequences

# # Get all sequences sorted by their total gap
# sorted_sequences = sorted(get_all_sequences(df['timed_matches'][:5].tolist()), key=lambda x: x[1])

# # Getting all sequences with the minimum gap (this will be the first sequence in the sorted list)
# min_gap = sorted_sequences[0][1]
# optimal_sequences = [seq for seq, gap in sorted_sequences if gap == min_gap]

# # Printing the sequences
# for seq in optimal_sequences:
#     print(seq)
#     print("----")
